# Generating figures from methods document

In [ ]:
import sqlite3
import random
import itertools
import subprocess
import os
import shlex
import re

In [ ]:
# path on rte
db_path = "/home/rte/data/db/arxiv_db_images.sqlite3"

# Here we import the sqlite3 database and create a cursor
db = sqlite3.connect(db_path)
c = db.cursor()

In [ ]:
# go through all of the retrieved SQL rows and format as an accreditation
# if html is True, format with <a href=####> for web usage
# input = [[author, title, date, identifier], [author, title, date, identifier],...[author, title, date, identifier]]

def format_credits(rows, html=False):
    accreditations = []

    empty_counter = 0
    
    # html = True
#     html = False

    # grab the details and check each
    for i, row in enumerate(rows):
        print(i)
        if row:
            s = row[0]
            start = "['"
            end = "']"
            author = s[s.find(start)+len(start):s.rfind(end)]
            author = author[:-2]
            print(author)

            title = row[1]
            # replace line breaks and double spaces
            title = title.replace("\n", "").replace("  "," ")
            print(title)

            date = row[2].split("-")[0]
            print(date)

            identifier = row[3]
            print(identifier)
        #     reg_exp = re.compile("/[^\d]\d{2}[^\d]/")
            longest_digits = max(re.findall(r'\d+', identifier), key = len)
            print(len(longest_digits))

            # if the identifier contains seven consecutive numbers, add a slash
            if len(longest_digits) == 7:
                print("----- regex match -----")
                reverse = identifier[::-1]
                print(reverse)
                identifier_reverse = reverse[:7] + "/" + reverse[7:]
                identifier = identifier_reverse[::-1]
            else:
                # otherwise we can leave the identifier how it is
                print("----- no match -----")
            print(identifier)
            url = "https://arxiv.org/abs/" + identifier
            print(url)
            print("*" * 20)
        #     print(row[0])

            # format string and append
            if html:
                fmt_str = '{}: {}, {}, <a href="{}">{}</a>'
                accreditations.append(fmt_str.format(author, title, str(date), url, url))  
            else:
                fmt_str = '{}: {}, {}, {}'
                accreditations.append(fmt_str.format(author, title, str(date), url, url))
        else:
            empty_counter += 1
            print("empty!")

    print("number of empty slots:", empty_counter)
    return accreditations

### Get random images

Selecting from October 2012 where possible, otherwise all of 2012

In [ ]:
# Get random images

name = "random_montage_144_oct2012"

sql = ("SELECT metadata.cat, images.path, images.filename, images.identifier, metadata.created, images.id, metadata.id "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.x != '' "
    "AND metadata.created BETWEEN date('2012-10-01') "
    "AND date('2012-10-31') ")

c.execute(sql, )
rows = c.fetchall()

In [ ]:
# Get images from cs.CV from October 2012

name = "csCV_montage_144_oct2012"

sql = ("SELECT metadata.cat, images.path, images.filename, images.identifier, metadata.created, images.id, metadata.id "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.x != '' "
    "AND metadata.created BETWEEN date('2012-10-01') "
    "AND date('2012-10-31') "
    "AND substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1) = ? ")

c.execute(sql, ("cs.CV", ))
rows = c.fetchall()

In [ ]:
# Get images from stat.ML

name = "statML_montage_144_oct2012"

sql = ("SELECT metadata.cat, images.path, images.filename, images.identifier, metadata.created, images.id, metadata.id "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.x != '' "
    "AND metadata.created BETWEEN date('2012-10-01') "
    "AND date('2012-10-31') "
    "AND substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1) = ? ")

c.execute(sql, ("stat.ML", ))
rows = c.fetchall()

In [ ]:
# Get all images from cs.AI from 2012 (whole year)

name = "csAI_montage_144_all2012"

sql = ("SELECT metadata.cat, images.path, images.filename, images.identifier, metadata.created, images.id, metadata.id "
    "FROM images "
    "LEFT JOIN metadata ON images.identifier = metadata.identifier "
    "WHERE images.x != '' "
    "AND metadata.created BETWEEN date('2012-01-01') "
    "AND date('2012-12-31') "
    "AND substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1) = ?")

c.execute(sql, ("cs.AI", ))
rows = c.fetchall()

In [ ]:
print(len(rows))
for row in rows[:3]:
    print(row)

In [ ]:
# shuffle first, then get ids and paths
# keep the seed as "4" so that results are reproducible
random.seed(4)
random.shuffle(rows)

meta_ids = []
ids = []
filepaths = []

for row in rows[:144]:
#     print(row[5])
    ids.append(row[5])
    meta_ids.append(row[6])
    path = row[1] + '/' + row[2]
#     print(path)
    filepaths.append(path)
print("ids length:",len(ids))

In [ ]:
print(meta_ids[:3])

In [ ]:
# get a montage of some images

outputname = ["/home/rte/documentation/data-samples/" + name + ".jpg"]
print(outputname)

# print the current working directory
os.chdir('/home/rte/re-imaging/sqlite-scripts/')
print(os.getcwd())

os.chdir('/home/rte/arXiv/')
print(os.getcwd())

prearg = shlex.split("-colorspace CMYK")
# arguments = shlex.split("-colorspace sRGB -background white -alpha background -trim +repage -flatten -geometry 240x240+2+2 -tile 12x /home/rte/documentation/data-samples/test_py_montage.jpg")

# for bigger montage of 12x12
arguments = shlex.split("-colorspace sRGB -background white -alpha background -geometry 240x240+2+2 -tile 12x")
# for smaller montage of 4x4
# arguments = shlex.split("-colorspace sRGB -background white -alpha background -geometry 480x480+2+2 -tile 4x")

# outputname = ["/home/rte/documentation/data-samples/random_montage_12x12_stat.ML.jpg"]
# outputname = ["/home/rte/documentation/data-samples/random_montage_nlin.CG.jpg"]
# print(arguments)

filelist = []
# this takes a slice of the larger shuffled list
for filepath in itertools.islice(filepaths, 0, 144):
#     print(filepath)
    # put the filepath into the list but add the directory, remove the dot
    # and also add [0] to only use the first page of multi-page image documents
    filelist.append("src_all" + filepath.replace('./','/') + '[0]')
           
# print(filelist)

# call the montage command and parse list of files and arguments
montage_cmd = ["montage"] + prearg + filelist + arguments + outputname

result = subprocess.Popen(montage_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = result.communicate()
print(out)
print(err)
print("subprocess finished")
print("-" * 40)


In [ ]:
### search for a specific set of entries and retrieve metadata
# this is for the image identifier
# author, title, date, identifier

rows = []

# sql = ("SELECT metadata.authors, metadata.title, metadata.created, metadata.identifier, images.id "
#     "FROM images "
#     "LEFT JOIN metadata ON images.identifier = metadata.identifier "
#     "WHERE images.id is ? ")

sql = ("SELECT metadata.authors, metadata.title, metadata.created, metadata.identifier "
    "FROM metadata "
    "WHERE metadata.id is ? ")

# iterate over the file_ids list and grab sql data
for file_id in meta_ids[:]:
    print("image parent article id:",file_id)
    c.execute(sql, (file_id, ))
    row = c.fetchall()
    rows.append(row[0])
    print(row[0])
#     print(len(row))
#     print(len(row[0]))
#     print(list(rows))

accreditations = format_credits(rows, False)

os.chdir("/home/rte/documentation/data-samples/")
print(os.getcwd())

f = open(name + ".txt", "w+")
for row in accreditations:
    f.write(str(row) + "\n")
f.close()